This notebook goal is to plot a figure using no custom function before we get to design and write useful ones

# Imports

In [139]:
# Third Parties
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np 

# Native

# Custom 
# /

# Classes

# Functions
from itertools import product

# Open file and prepare for plotting

In [96]:
articles = pd.read_csv(
    '../../data/2025-01-07-2024-10-29-quinquadef4-neat-abstract-bert.csv'
)

revues = pd.read_csv(
    '../../data/Classification revues - Feuille 1.csv'
)

/var/folders/xs/d90v1vkn16db_7z493h6kwt40000gn/T/ipykernel_7059/3563665539.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  articles = pd.read_csv(


In [97]:
articles.columns

Index(['iddef', 'titre', 'soustitre', 'auteurs', 'annee', 'revue', 'num_titre',
       'num_num', 'abstract', 'url_art', 'auteurs_fam', 'auteurs_prenom',
       'source', 'n_auteurs', 'femme', 'annee5', 'majo_femmes', 'sexe3',
       'tag_race', 'tag_inclusif', 'bert_pronoms', 'bert_quanti', 'bert_quali',
       'bert_genre', 'bert_genre_stat', 'bert_classe_large',
       'bert_classe_stricte', 'bert_genre_pas_stat', 'bert_monostricte',
       'bert_interstricte_2plus', 'bert_interstricte_3',
       'bert_interstricte_GC', 'bert_interstricte_GR', 'bert_interstricte_CR',
       'bert_interstricte_cl', 'bert_interstricte_cl2', 'bert_monolarge',
       'bert_interlarge_2plus', 'bert_interlarge_3', 'bert_interlarge_GC',
       'bert_interlarge_GR', 'bert_interlarge_CR', 'bert_interlarge_cl',
       'bert_interlarge_cl2'],
      dtype='object')

In [98]:
revues.columns

Index(['revue', 'Dominante', 'Aréale', 'Anthropo', 'Demo', 'Eco', 'Genre',
       'Géographie', 'Histoire', 'SIC', 'Sociologie', 'SciencePo'],
      dtype='object')

## Doing non-sense because i am stupid

In [99]:
# Transforming cells, where NaN -> False and where x -> True
# revues.replace({np.nan: False, 'x' : True})
revues = revues.fillna(False)
revues = revues.replace({'x' : True})
revues = revues.astype({
    'revue'         : str,
    'Dominante'     : str,
    'Aréale'        : bool,
    'Anthropo'      : bool,
    'Demo'          : bool,
    'Eco'           : bool,
    'Genre'         : bool,
    'Géographie'    : bool,
    'Histoire'      : bool,
    'SIC'           : bool,
    'Sociologie'    : bool,
    'SciencePo'     : bool
})
revues.loc[:,'Dominante'] = revues.loc[:,'Dominante'].replace({'Économie' : 'Economie'})

/var/folders/xs/d90v1vkn16db_7z493h6kwt40000gn/T/ipykernel_7059/1780883791.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  revues = revues.replace({'x' : True})


In [100]:
CAT = [
    'Aréale'    ,
    'Anthropologie'  ,
    'Démographie'      ,
    'Economie'       ,
    'Genre'     ,
    'Géographie',
    'Histoire'  ,
    'SIC'       ,
    'Sociologie',
    'Science politique' 
]

df = pd.DataFrame({
    "annee"         : [],
    'Aréale'        : [],
    'Anthropologie' : [],
    'Démographie'   : [],
    'Economie'      : [],
    'Genre'         : [],
    'Géographie'    : [],
    'Histoire'      : [],
    'SIC'           : [],
    'Sociologie'    : [],
    'Science politique'         : [],
    'Autre interdisciplinaire'  : []
})


In [101]:
revues.head()

,revue,Dominante,Aréale,Anthropo,Demo,Eco,Genre,Géographie,Histoire,SIC,Sociologie,SciencePo
0,Les Études Sociales,Autre interdisciplinaire,False,False,False,False,False,False,True,False,True,False
1,Ethnologie française,Anthropologie,False,True,False,False,False,False,False,False,True,False
2,Hommes & Migrations,Anthropologie,False,True,False,False,False,False,False,False,False,False
3,Journal des anthropologues,Anthropologie,False,True,False,False,False,False,False,False,False,False
4,L’Homme,Anthropologie,False,True,False,False,False,False,False,False,False,False


In [102]:
df

,annee,Aréale,Anthropologie,Démographie,Economie,Genre,Géographie,Histoire,SIC,Sociologie,Science politique,Autre interdisciplinaire


In [103]:
unknown_revues = []
unknown_categories = []
for i in range(len(articles)): 
    # retrieve the publication year and the name of the revue
    revue_i : str = articles.loc[i,'revue']
    annee_i : int = int(articles.loc[i,'annee'])

    # find if the category
    try : 
        found_categorie : str = revues.loc[
                                    revues.revue == revue_i,
                                    'Dominante'].item()
    except:
        unknown_revues.append(revue_i)
        pass

    if found_categorie in CAT : 
        cat_label : str = found_categorie
    else : 
        if found_categorie != 'Autre interdisciplinaire' : 
            unknown_categories.append(found_categorie)
            
        cat_label : str = 'Autre interdisciplinaire'

    # Make sure the row for the year exists
    if annee_i not in df['annee'].values:
        # if not, create it
        df.loc[len(df),:] = [annee_i] + [0] * (len(df.columns) - 1)
    
    # add data to df
    df.loc[
        df['annee'] == annee_i,
        cat_label
    ] += 1
    

In [104]:
set(unknown_revues)

{'Commentaire', 'Éducation & formations'}

In [105]:
set(unknown_categories)

set()

In [112]:
df = df.astype({
    "annee"         : int,
    'Aréale'        : int,
    'Anthropologie' : int,
    'Démographie'   : int,
    'Economie'      : int,
    'Genre'         : int,
    'Géographie'    : int,
    'Histoire'      : int,
    'SIC'           : int,
    'Sociologie'    : int,
    'Science politique'         : int,
    'Autre interdisciplinaire'  : int
})
df = df.sort_values('annee')
df.index = np.arange(0,len(df))
df

,annee,Aréale,Anthropologie,Démographie,Economie,Genre,Géographie,Histoire,SIC,Sociologie,Science politique,Autre interdisciplinaire
0,2001,56,80,4,29,54,15,153,87,165,200,170
1,2002,73,82,25,61,57,14,185,131,157,171,185
2,2003,146,79,23,95,72,15,176,145,200,191,253
3,2004,122,88,20,94,67,36,188,162,197,206,348
4,2005,104,87,17,109,69,45,189,130,224,159,411
5,2006,153,93,21,153,81,48,187,144,203,170,492
6,2007,139,108,36,180,64,37,238,127,285,239,524
7,2008,186,106,32,164,67,47,265,170,334,254,468
8,2009,244,91,26,120,89,39,277,184,334,236,524
9,2010,233,108,50,146,83,45,279,181,344,271,569


In [113]:
df.to_csv('../../data/checkpoints/categories_DRAFT.csv')

## Being smart starts now

In [121]:
unknown_revues = []
unknown_categories = []

found_categorie : list[str] = []

for i in range(len(articles)): 
    # retrieve the publication year and the name of the revue
    revue_i : str = articles.loc[i,'revue']

    # find if the category
    try : 
        found_categorie_i : str = revues.loc[
                                    revues.revue == revue_i,
                                    'Dominante'].item()
    except:
        unknown_revues.append(revue_i)
        found_categorie_i = "NaN"

    if found_categorie_i in CAT : 
        cat_label : str = found_categorie_i
    if found_categorie_i == "NaN":
        cat_label : str = "NaN"
    else : 
        if found_categorie_i != 'Autre interdisciplinaire' : 
            unknown_categories.append(found_categorie)
            
        cat_label : str = 'Autre interdisciplinaire'
    found_categorie.append(found_categorie_i)


In [120]:
unknown_revues

['Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Éducation & formations',
 'Éducation & formations',
 'Éducation & formations',
 'Éducation & formations',
 'Éducation & formations',
 'Éducation & formations']

In [122]:
set(found_categorie)

{'Anthropologie',
 'Aréale',
 'Autre interdisciplinaire',
 'Démographie',
 'Economie',
 'Genre',
 'Géographie',
 'Histoire',
 'NaN',
 'SIC',
 'Science politique',
 'Sociologie'}

In [125]:
articles['Discipline'] = found_categorie

In [129]:
articles.loc[articles.Discipline == "NaN",['revue','Discipline']]

,revue,Discipline
5916,Commentaire,NaN
5917,Commentaire,NaN
5918,Commentaire,NaN
5919,Commentaire,NaN
5920,Commentaire,NaN
5921,Commentaire,NaN
5922,Commentaire,NaN
5923,Commentaire,NaN
5924,Commentaire,NaN
5925,Commentaire,NaN


In [132]:
# Create usable df
df = articles.loc[
    articles.Discipline != "NaN",
    ["annee", "bert_genre", "Discipline"]
]
df = df.sort_values('annee')
df.index = np.arange(0,len(df))
df.head()

,annee,bert_genre,Discipline
0,2001,False,Histoire
1,2001,False,Science politique
2,2001,False,Science politique
3,2001,False,Science politique
4,2001,False,Science politique


In [133]:
set(df.Discipline)

{'Anthropologie',
 'Aréale',
 'Autre interdisciplinaire',
 'Démographie',
 'Economie',
 'Genre',
 'Géographie',
 'Histoire',
 'SIC',
 'Science politique',
 'Sociologie'}

In [149]:
# create the df to plot for all the categories
# NOTE mind the nan in bert_genre
annee, discipline, pourcentage = [],[],[]

for y,cat in product(set(df.annee), set(df.Discipline)):
    annee.append(y)
    discipline.append(cat)
    pourcentage.append(
        df.loc[
            (df.annee == y) & (df.Discipline == cat),
            "bert_genre"
        ].mean() * 100
    )

# Add the "Toutes" groupe
for y in set(df.annee):
    annee.append(y)
    discipline.append("Toutes")
    pourcentage.append(
        df.loc[
            (df.annee == y),
            "bert_genre"
        ].mean() * 100
    )

# Add the "Toutes Sauf Genre" groupe
for y in set(df.annee):
    annee.append(y)
    discipline.append("Toutes sauf Genre")
    pourcentage.append(
        df.loc[
            (df.annee == y) & (df.Discipline != 'Genre'),
            "bert_genre"
        ].mean() * 100
    )

In [153]:
# running mean : https://stackoverflow.com/questions/13728392/moving-average-or-running-mean
running_mean_window_lenght = 2
window = np.ones(running_mean_window_lenght) / running_mean_window_lenght

# TODO deal with running mean

dfPlot = pd.DataFrame({
    "annee" : annee,
    "Discipline" : discipline,
    "pourcentage" : pourcentage,
})
dfPlot

ValueError: All arrays must be of the same length

# Plotting

The plotting section is first going to turn all the data into a usable data frame. We are going to use the following logic. : 
> Each row is a data, it will contain the year, percentage and any tag used for the actual plotting section

## Actually plotting

In [151]:
fig = px.line(
    dfPlot,
    x = 'annee', y = 'pourcentage',
    color = 'Discipline',
    labels = {
            'Discipline' : 'Dominante de la revue',
        }
)

In [152]:
fig

### Customisation

In [195]:
# Setting title and axis names
fig = fig.update_layout(dict(
    title = dict(
            text = ("Évolution de la proportion d'articles publiés invoquant le "
                    "thème du genre (def extensive et statistique)"),
             automargin = True, 
             pad = dict(t = 10, b = 10 , r = 10, l = 10)),
    xaxis = dict(title = "Année de publication"),
    yaxis = dict(
        title = "Part des articles qui invoquent le concept du genre",
        ticksuffix = "%"),    
))

In [213]:
# https://plotly.com/python/reference/layout/xaxis/
# customizing the x axis
fig = fig.update_layout(
    xaxis = dict(
        showline = True,
        linewidth = 2, 
        linecolor = 'rgb(20,20,20)',
        ticks = "outside",           # "outside", "inside", ""
        showticklabels = True,
        tickangle = 0,
        tickfont = dict(
            family = "Arial", size = 12, color = "rgb(80,80,80)"
            ),
        tickvals = [2004,2008,2012,2016,2020],
        showgrid = False,
    )
)

# customizing the y axis
fig = fig.update_layout(
    yaxis = dict(
        showline = False,
        ticks = "outside",
        showticklabels = True,
        tickangle = 0,
        tickfont = dict(
            family = "Arial", size = 12, color = "rgb(80,80,80)"
            ),
        tickvals = [0,10,20],
        tickwidth = 1,
        showgrid = True,
        gridcolor = 'rgb(80,80,80)',
        gridwidth = 1,

        autorange = False,
        range = [-0.001,30]
    )
)

# changing fig background color : 
fig = fig.update_layout(dict(
    plot_bgcolor='rgb(255, 255, 255)',
    paper_bgcolor='rgb(255, 255, 255)',
))

In [214]:
# Not implemented but might be interesting
# fig.update_layout(dict(
#     hovermode = "closest",
#     xaxis = dict(showspikes = True)
# ))

In [215]:
# Adding a dropdown menu https://programminghistorian.org/en/lessons/interactive-visualization-with-plotly#adding-animations-dropdown-bars

fig = fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            pad = dict(r = 10, t = 10, b = 10, l = 10),  #Pad autour de l'objet
            showactive = True, 
            xanchor = "left", yanchor = "top",
            x = .05, y = 1.1,
            buttons=list(
                [ 
                    dict(
                        label="Toutes les courbes",                     
                        method="update",
                        args=[
                            {
                                "visible": [
                                    True, True,
                                    True, True,
                                    True, True,
                                    True, True
                                    ]
                            },  # This 'view' show all three types of crime
                            {
                                "title": ("Évolution de la proportion d'articles"
                                          "publiés invoquant le thème du genre "
                                          "(def extensive et statistique)"),
                            },
                        ],
                    ),
                    dict(
                        label="Toutes les revues", 
                        method="update",
                        args=[
                            {
                                "visible": [
                                    True, True,
                                    False, False,
                                    False, False,
                                    False, False
                                    ]
                            },  # This 'view' show all three types of crime
                            {
                                "title": ("Évolution de la proportion d'articles"
                                          " publiés invoquant le thème du genre "
                                          " (def extensive et statistique)\n"
                                          " Toutes les revues confondues"),
                            },
                        ],
                    ),
                    dict(
                        label="Revues spéciales", 
                        method="update",
                        args=[
                            {
                                "visible": [
                                    False, False,
                                    True, True,
                                    True, True,
                                    True, True
                                    ]
                            },  # This 'view' show all three types of crime
                            {
                                "title": ("Évolution de la proportion d'articles"
                                          " publiés invoquant le thème du genre "
                                          " (def extensive et statistique)\n"
                                          " ARSS, groupe TYPE et GENERALE"),
                            },
                        ],
                    ),
                    dict(
                        label="Définition extensive uniquement", 
                        method="update",
                        args=[
                            {
                                "visible": [
                                    True, False,
                                    True, False,
                                    True, False,
                                    True, False
                                    ]
                            },  # This 'view' show all three types of crime
                            {
                                "title": ("Évolution de la proportion d'articles"
                                          " publiés invoquant le thème du genre "
                                          " (def extensive)"),
                            },
                        ],
                    ),
                    dict(
                        label="Définition statistique uniquement", 
                        method="update",
                        args=[
                            {
                                "visible": [
                                    False, True,
                                    False, True,
                                    False, True,
                                    False, True
                                    ]
                            },  # This 'view' show all three types of crime
                            {
                                "title": ("Évolution de la proportion d'articles"
                                          " publiés invoquant le thème du genre"
                                          " (def statistique)"),
                            },
                        ],
                    ),
                ]
            )
        )
    ]
)
fig = fig.update_layout(dict(margin = dict(autoexpand = True)))
# TODO check dimension des boutons 

In [216]:
# setting the dimentions
fig = fig.update_layout(dict(height = 600, width = 1000))

In [217]:
# Customise the legend
fig = fig.update_layout(dict(
    legend = dict(
        bgcolor = 'rgba(230,230,230,0.6)',
        title = dict(text = 'Données exposées', side = "top center",
                     font = dict(color = "red", variant = "all-small-caps")),
        itemdoubleclick = "toggleothers", itemclick = "toggle",
        xanchor = "left", yanchor = "bottom",
        x = 1.1, y = .5,
        # width = 100, height = 100
    )
))

In [253]:
# Fixing the hover information
fig = fig.update_traces(hovertemplate ='%{y:.2f}\\%')
# fig = fig.update_layout(hovermode="x")
fig = fig.update_layout(hovermode="x unified")
# NOTE might be worth reading
# https://stackoverflow.com/questions/59057881/how-to-customize-hover-template-on-with-what-information-to-show

# Customising hover
fig = fig.update_layout(
    hoverlabel = dict(
        namelength = 0,
        bgcolor = 'rgba(255,255,255,0.8)',
        font = dict(
            size = 12,
            family = "Arial"),
        grouptitlefont = dict(
            color = "red")
        # NOTE On peut créer des groupes, 
        # TODO à creuser

))

In [254]:
# change the names of the plots 
names_of_curves = [
    'Toutes revues confondues (ext)',
    'Toutes revues confondues (stat)',
    'ARSS (ext)',
    'ARSS (stat)',
    'Revue TYPE (ext)',
    'Revue TYPE (stat)',
    'Revue GENERALE (ext)',
    'Revue GENERALE (stat)',
]

for name, plot in zip(names_of_curves, fig.data):
    plot.update(name = name)

# change the colours of the plots 
colours_of_curves = [
    '#3E50B6',
    '#3E50B6',
    '#29BD00',
    '#29BD00',
    '#F202D6',
    '#F202D6',
    '#F46E01',
    '#F46E01',
]

for name,colour, plot in zip(names_of_curves,colours_of_curves, fig.data):
    plot.update(name = name)
    plot.update(line_color = colour)
    plot.update(line_width = 2)

# The plot

In [255]:
fig.show()#{'responsive' : False})

In [223]:
fig.data[0]['onclick']

PlotlyKeyError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'onclick'

Did you mean "line"?

    Valid properties:
        alignmentgroup
            Set several traces linked to the same position axis or
            matching axes to the same alignmentgroup. This controls
            whether bars compute their positional range dependently
            or independently.
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
   

In [ ]:
double click pour griser plutôt que faire disparaitre 

In [56]:
with open('test.html','w') as file:
    file.write(fig.to_html())

In [ ]:
# https://plotly.com/python/interactive-html-export/#full-parameter-documentation
fig.write_html('test_.html', include_plotlyjs = False)

In [ ]:
fig.data[0].update(name = 'A')

In [ ]:
'width' in fig.data[0]

In [180]:
import plotly.graph_objs as go

fig=go.Figure()
fig.add_scatter(y=[1, 1, 1],  line={'color': 'red'},name='firsttrace')
fig.add_scatter(y=[2, 2, 2],  line={'color': 'blue'},name='Secondttrace')
fig.add_scatter(y=[3, 3, 3],  line={'color': 'purple'},name='Thirdtrace')
fig.add_scatter(y=[4, 4, 4],  line={'color': 'gray'},name='Forthtrace')


button = [dict(
    method='update',  
    args=[
        {
            'mode':[
                'markers+lines', 
                'markers+lines', 
                'markers+lines', 
                'markers+lines'
            ],
            'marker':[
                {'color': ['pink', 'red', 'green'], 'size':[6,15,23], 'line':{'color':'black'}},
                {'color': ['orange', 'blue', 'black'], 'size':[23,11,8], 'line':{'color':'black'}},
                {'color': ['red', 'white', 'gray'], 'size':[1,15,30], 'line':{'color':'black'}},
                {'color': ['green', 'purple', 'magenta'], 'size':[23,11,8], 'line':{'color':'black'}},


            ],
            'line':[
                {'color':'crimson'},
                {'color':'blue', 'width':8},
                {'color':'green'},
                {'color':'white'},
            ]
        },
    ],
    label='button'
)]
    

fig.layout.updatemenus = [{'buttons': button}]
from plotly.offline import iplot
iplot(fig)
fig.show()

In [ ]:
from plotl